# Experiment 007: Backward Propagation

For each N from 200 down to 2:
- Take the N configuration
- Try removing each tree one at a time
- If the resulting (N-1) configuration has smaller bounding box than current (N-1), replace it

This propagates good packings from larger N to smaller N.

In [ ]:
import numpy as np
import pandas as pd
from decimal import Decimal, getcontext
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
import copy
import time

getcontext().prec = 25
scale_factor = Decimal("1e15")

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

print("Setup complete")

In [ ]:
class ChristmasTree:
    """Christmas tree with Decimal precision"""
    def __init__(self, center_x="0", center_y="0", angle="0"):
        self.center_x = Decimal(str(center_x))
        self.center_y = Decimal(str(center_y))
        self.angle = Decimal(str(angle))
        self._update_polygon()
    
    def _update_polygon(self):
        vertices = []
        for tx, ty in zip(TX, TY):
            vertices.append((float(Decimal(str(tx)) * scale_factor), 
                           float(Decimal(str(ty)) * scale_factor)))
        initial_polygon = Polygon(vertices)
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(
            rotated,
            xoff=float(self.center_x * scale_factor),
            yoff=float(self.center_y * scale_factor)
        )
    
    def clone(self):
        return ChristmasTree(str(self.center_x), str(self.center_y), str(self.angle))

def calculate_side(trees):
    """Calculate bounding box side length"""
    all_polygons = [t.polygon for t in trees]
    bounds = unary_union(all_polygons).bounds
    return Decimal(str(max(bounds[2] - bounds[0], bounds[3] - bounds[1]))) / scale_factor

def has_collision(trees):
    """Check for collisions"""
    for i, t1 in enumerate(trees):
        for j, t2 in enumerate(trees):
            if i < j:
                if t1.polygon.intersects(t2.polygon) and not t1.polygon.touches(t2.polygon):
                    return True
    return False

print("Classes defined")

In [ ]:
# Load baseline submission
baseline_df = pd.read_csv('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv')

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return val[1:]
    return str(val)

def load_trees_for_n(df, n):
    prefix = f"{n:03d}_"
    rows = df[df['id'].str.startswith(prefix)]
    trees = []
    for _, row in rows.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        trees.append(ChristmasTree(x, y, deg))
    return trees

# Load all configurations
print("Loading all configurations...")
configs = {}
sides = {}
for n in range(1, 201):
    trees = load_trees_for_n(baseline_df, n)
    configs[n] = trees
    sides[n] = calculate_side(trees)
    if n % 50 == 0:
        print(f"  Loaded N={n}, side={sides[n]:.6f}")

print(f"\nLoaded {len(configs)} configurations")

In [ ]:
# Calculate baseline total score
baseline_total = sum(float(sides[n])**2 / n for n in range(1, 201))
print(f"Baseline total score: {baseline_total:.6f}")

In [ ]:
# Backward propagation
print("\nRunning backward propagation...")
print("="*60)

improvements = []
new_configs = {n: [t.clone() for t in configs[n]] for n in range(1, 201)}
new_sides = {n: sides[n] for n in range(1, 201)}

for n in range(200, 1, -1):
    target_n = n - 1
    current_best_side = new_sides[target_n]
    best_tree_to_remove = None
    best_new_side = current_best_side
    
    # Try removing each tree from N configuration
    for tree_idx in range(n):
        # Create candidate by removing tree at tree_idx
        candidate = [t.clone() for i, t in enumerate(new_configs[n]) if i != tree_idx]
        
        # Check no collisions (should be fine since we're removing a tree)
        if has_collision(candidate):
            continue
        
        candidate_side = calculate_side(candidate)
        
        if candidate_side < best_new_side:
            best_new_side = candidate_side
            best_tree_to_remove = tree_idx
    
    # If we found an improvement, update
    if best_tree_to_remove is not None and best_new_side < current_best_side:
        improvement = float(current_best_side)**2 / target_n - float(best_new_side)**2 / target_n
        improvements.append({
            'target_n': target_n,
            'source_n': n,
            'removed_tree': best_tree_to_remove,
            'old_side': float(current_best_side),
            'new_side': float(best_new_side),
            'score_improvement': improvement
        })
        
        # Update the configuration
        new_configs[target_n] = [t.clone() for i, t in enumerate(new_configs[n]) if i != best_tree_to_remove]
        new_sides[target_n] = best_new_side
        
        print(f"N={target_n}: Improved! side {current_best_side:.6f} -> {best_new_side:.6f} (removed tree {best_tree_to_remove} from N={n})")
    
    if n % 20 == 0:
        print(f"  Processed N={n}...")

print("\nBackward propagation complete!")

In [ ]:
# Summary of improvements
print("\n" + "="*60)
print("BACKWARD PROPAGATION RESULTS")
print("="*60)

if improvements:
    print(f"\nFound {len(improvements)} improvements:")
    total_improvement = 0
    for imp in improvements:
        print(f"  N={imp['target_n']}: {imp['old_side']:.6f} -> {imp['new_side']:.6f} (improvement: {imp['score_improvement']:.9f})")
        total_improvement += imp['score_improvement']
    
    print(f"\nTotal score improvement: {total_improvement:.9f}")
    
    # Calculate new total score
    new_total = sum(float(new_sides[n])**2 / n for n in range(1, 201))
    print(f"\nBaseline score: {baseline_total:.6f}")
    print(f"New score: {new_total:.6f}")
    print(f"Improvement: {baseline_total - new_total:.9f}")
else:
    print("\nNo improvements found.")
    print("The baseline configurations are already optimal for backward propagation.")

In [ ]:
# If improvements found, save the new submission
if improvements:
    print("\nSaving improved submission...")
    
    def to_str(val):
        return f"s{val}"
    
    rows = []
    for n in range(1, 201):
        for i, tree in enumerate(new_configs[n]):
            rows.append({
                'id': f"{n:03d}_{i}",
                'x': to_str(tree.center_x),
                'y': to_str(tree.center_y),
                'deg': to_str(tree.angle)
            })
    
    new_df = pd.DataFrame(rows)
    new_df.to_csv('/home/code/experiments/007_backward_propagation/submission.csv', index=False)
    new_df.to_csv('/home/submission/submission.csv', index=False)
    print("Saved to /home/submission/submission.csv")
    
    # Verify
    print(f"\nVerification: {len(new_df)} rows")
else:
    # Copy baseline to submission
    import shutil
    shutil.copy('/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025-csv/santa-2025.csv',
                '/home/submission/submission.csv')
    print("\nNo improvements - using baseline submission")